In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


## 1. Set hyper parameters

In [2]:
lr = 0.001
train_epoch = 15
bsize = 100

tf.random.set_seed(777)

#### + Create checkpoint directory

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_cnn_ensemble'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
# os.makedirs(checkpoint_dir, exist_ok=True)

ckpt_prefix = os.path.join(checkpoint_dir, model_dir_name)

## 2. Make a data pipelining

In [4]:
mnist = keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype(np.float32) / 255.
X_train = np.expand_dims(X_train, axis=-1)
y_train = to_categorical(y_train, num_classes=10)

X_test = X_test.astype(np.float32) / 255.
X_test = np.expand_dims(X_test, axis=-1)
y_test = to_categorical(y_test, num_classes=10)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).\
                    shuffle(buffer_size=100000).batch(batch_size=bsize)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).\
                    batch(batch_size=bsize)

## 3. Build a neural network model

In [6]:
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(filters=32, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool1 = keras.layers.MaxPool2D(pool_size=2, padding='same',
                                            strides=2)
        self.conv2 = keras.layers.Conv2D(filters=64, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool2 = keras.layers.MaxPool2D(pool_size=2, padding='same',
                                            strides=2)
        self.conv3 = keras.layers.Conv2D(filters=128, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool3 = keras.layers.MaxPool2D(pool_size=2, padding='same',
                                            strides=2)
        self.pool3_flat = keras.layers.Flatten()
        
        self.dense4 = keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.drop4 = keras.layers.Dropout(rate=0.4)
        self.dense5 = keras.layers.Dense(units=10)
        pass
    
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

models = []
num_models = 3
for i in range(num_models):
    model = MNISTModel()
    models.append(model)

## 4. Define a loss function

In [7]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean( 
                keras.losses.categorical_crossentropy(
                    y_true=labels, y_pred=logits, from_logits=True))
    return loss

## 5. Calculate a gradient

In [9]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient( loss, model.variables )

## 6. Select an optimizer

In [10]:
optimizer = keras.optimizers.Adam(learning_rate=lr)

## 7. Define a metric for model's performance

In [11]:
def evaluate(models, images, labels):
    prediction = np.zeros_like(labels)
    for model in models:
        logits = model(images, training=False)
        predictions += logits
    correct_prediction = tf.equal( x=tf.argmax(predictions, 1),
                                   y=tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))    
    return accuracy

## 8. Make a checkpoint for saving

In [12]:
checkpoints = []
for i in range(num_models):
    checkpoints.append(tf.train.Checkpoint(cnn=models[i]))

## 9. Train and Validate a neural network model